In [1]:
import tensorflow as tf
import pandas as pd
from modules import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

2023-12-25 00:32:33.634727: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-25 00:32:33.636738: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-25 00:32:33.663556: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-25 00:32:33.663583: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-25 00:32:33.664327: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

#### Load The Data
The user content is composed of engineered features. A per genre average rating is computed per user. Additionally, a user id, rating count and rating average are available but not included in the training or prediction content. They are carried with the data set because they are useful in interpreting data.

The training set consists of all the ratings made by the users in the data set. Some ratings are repeated to boost the number of training examples of underrepresented genre's. The training set is split into two arrays with the same number of entries, a user array and a destination/item array.

Below, let's load and display some of the data.

In [2]:
item_train, user_train, y_train, item_features, user_features, item_vecs, destination_dict = load_data()
num_user_features = user_train.shape[1] - 3 # remove remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1 # remove destination id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items

print(f"item: {len(item_train)}, user: {len(user_train)}, y:  {len(y_train)}, item features: {len(item_features)}, user features: {len(user_features)}, item vectors: {len(item_vecs)}, destination dict: {len(destination_dict)}")

item: 437, user: 437, y:  437, item features: 9, user features: 9, item vectors: 437, destination dict: 437


In [3]:
pprint_train(user_train, user_features, uvs,  u_s, maxcount=5)

,0,1,2,3,4,5,6,7,8
0,[Use r_Id],[Rating _Count],[Rating_ Average],Bah ari,Bud aya,Cagar Alam,Pusat Perbelanjaan,Taman Hiburan,Tempat Ibadah
1,1,30,3.366667,2.0,3.5,3.444444,3.0,3.8,2.0
2,2,29,3.241379,2.0,3.444444,2.8,0.0,3.5,4.0
3,3,30,3.366667,4.333333,3.230769,4.0,0.0,2.9,5.0
4,4,26,3.423077,5.0,3.083333,3.75,4.0,3.5,0.0
5,5,33,3.363636,3.5,3.285714,2.888889,5.0,3.461538,5.0


Some of the user and item/destination features are not used in training. In the table above, the features in brackets "[]" such as the "user id", "rating count" and "rating ave" are not included when the model is trained and used.

In [4]:
pprint_train(item_train, item_features, ivs,  i_s, maxcount=5)

,0,1,2,3,4,5,6,7,8
0,[Plac e_Id],Price,Place_ Ratings,Bah ari,Bud aya,Cagar Alam,Pusat Perbelanjaan,Taman Hiburan,Tempat Ibadah
1,1,20000,3.722222,0.0,1.0,0.0,0.0,0.0,0.0
2,2,0,2.84,0.0,1.0,0.0,0.0,0.0,0.0
3,3,270000,2.526316,0.0,0.0,0.0,0.0,1.0,0.0
4,4,10000,2.857143,0.0,0.0,0.0,0.0,1.0,0.0
5,5,94000,3.52,0.0,0.0,0.0,0.0,1.0,0.0


#### Preparing the training data
Below, the inverse_transform is also shown to produce the original inputs. We'll scale the target ratings using a Min Max Scaler which scales the target to be between -1 and 1

In [5]:
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


To allow us to evaluate the results, we will split the data into training and test sets
Here we will use sklean train_test_split to split and shuffle the data. Note that setting the initial random state to the same value ensures item, user, and y are shuffled identically

In [6]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"destination/item training data shape: {item_train.shape}")
print(f"destination/item test data shape: {item_test.shape}")

destination/item training data shape: (349, 9)
destination/item test data shape: (88, 9)


#### Neural Network for content-based filtering
It will have two networks that are combined by a dot product. We construct the two networks. In this example, they will be identical. Note that these networks do not need to be the same. If the user content was substantially larger than the destination content, you might elect to increase the complexity of the user network relative to the destination network. In this case, the content is similar, so the networks are the same

In [7]:
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
])

item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=num_user_features)
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=num_item_features)
vd = item_NN(input_item)
vd = tf.linalg.l2_normalize(vd, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vd])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 6)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 8)]                  0         []                            
                                                                                                  
 sequential (Sequential)     (None, 32)                   38816     ['input_1[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 32)                   39328     ['input_2[0][0]']             
                                                                                              

We use a mean squared error loss and an Adam optimizer.

In [8]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt, loss=cost_fn)

In [9]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=100)

Epoch 1/100
11/11 [==============================] - 1s 2ms/step - loss: 0.0892
Epoch 2/100
11/11 [==============================] - 0s 3ms/step - loss: 0.0314
Epoch 3/100
11/11 [==============================] - 0s 2ms/step - loss: 0.0212
Epoch 4/100
11/11 [==============================] - 0s 3ms/step - loss: 0.0155
Epoch 5/100
11/11 [==============================] - 0s 2ms/step - loss: 0.0123
Epoch 6/100
11/11 [==============================] - 0s 2ms/step - loss: 0.0131
Epoch 7/100
11/11 [==============================] - 0s 2ms/step - loss: 0.0102
Epoch 8/100
11/11 [==============================] - 0s 2ms/step - loss: 0.0086
Epoch 9/100
11/11 [==============================] - 0s 2ms/step - loss: 0.0070
Epoch 10/100
11/11 [==============================] - 0s 3ms/step - loss: 0.0059
Epoch 11/100
11/11 [==============================] - 0s 3ms/step - loss: 0.0052
Epoch 12/100
11/11 [==============================] - 0s 3ms/step - loss: 0.0045
Epoch 13/100
11/11 [=================

In [10]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

3/3 [==============================] - 0s 2ms/step - loss: 0.0090


0.008993084542453289

#### Predictions
Below, use the model to make predictions in a number of circumstances
we'll create a user and have the model suggest destination for that user. After you have tried this on the example user content, feel free to change the user content to match your own preferences and see what the model suggests. Note that ratings are between 0.5 and 5.0, inclusive, in half-step increments.

In [11]:
new_id = 5000
new_rating_count = 20.0
new_rating_ave = 10.0
new_bahari = 0.0
new_budaya = 5.0
new_cagar_alam = 5.0
new_pusat_perbelanjaan = 5.0
new_taman_hiburan = 0.0
new_tempat_ibadah = 0.0

user_vec = np.array([[new_id, new_rating_count, new_rating_ave,
                      new_bahari, new_budaya, new_cagar_alam, new_pusat_perbelanjaan,
                      new_taman_hiburan, new_tempat_ibadah]])

In [12]:
# generate and replicate the user vector to match the number destination in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display

print_pred_destination(sorted_ypu, sorted_items, destination_dict, maxcount = 100)

14/14 [==============================] - 0s 1ms/step


,0,1,2,3,4
0,y_p,id,rating ave,name,category
1,3.0,179,3.1,Candi Ratu Boko,Budaya
2,3.0,325,2.9,Saung Angklung Mang Udjo,Budaya
3,3.0,39,3.3,Museum Macan (Modern and Contemporary Art in N...,Budaya
4,3.0,331,2.8,Kyotoku Floating Market,Budaya
...,...,...,...,...,...
96,3.0,43,2.6,The Escape Hunt,Taman Hiburan
97,3.0,35,3.0,Grand Indonesia Mall,Pusat Perbelanjaan
98,3.0,3,2.5,Dunia Fantasi,Taman Hiburan
99,3.0,45,2.7,Jakarta Aquarium dan Safari,Taman Hiburan
